## Pre-Training [`GPT-Neo`](https://github.com/huggingface/transformers/blob/main/src/transformers/models/gpt_neo/modeling_gpt_neo.py) on [`TinyStories`](https://huggingface.co/datasets/roneneldan/TinyStories)

This script shows how to pre-train both models. I put them in one notebook because the majority of the code is shared; but you may want to separate the logic per model. 

In [7]:
# %pip install torch wandb transformers[torch] datasets tqdm 
%load_ext autoreload
%autoreload 2 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
import os
import sys
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # or "0,1" for multiple GPUs


import wandb; wandb.login()

# TODO: Some gpu error, this fixes it but ask aral for the details
import torch
torch.use_deterministic_algorithms(False)

small_dataset = True
if len(sys.argv) > 1 and sys.argv[1] and sys.argv[1] == "true":
    small_dataset = True
elif len(sys.argv) > 1 and sys.argv[1] and sys.argv[1] == "false":
    small_dataset = False


from transformers import (
    GPTNeoForCausalLM, GPTNeoConfig, GPT2TokenizerFast, set_seed
)

#### Models 
We consider GPT-Neo and BERT as base transformer architectures. This consists of the following blocks linked via residual connections:

- Embeddings: Map one-hot (sparse) token vectors to a dense vector of length `hidden_size`. Add positional encoding. 
- $n$ Blocks: Contain self-attention and FFNs.
- Head: Map hidden state back to a useful output. 

This specifies some of the model-related hyperparameters. I chose them based on what achieved reasonable performance in the [TinyStories paper](https://arxiv.org/abs/2305.07759), while also being feasible to train on our limited compute budgets. 

In [9]:
config_gpt = dict(

    # EMBEDDING PARAMETERS
    vocab_size              = 10_000,   # number of tokens in the vocabulary 
    hidden_size             = 516,      # embedding size (vector length) of each token 
    max_position_embeddings = 512,      # maximum sequence length (context window)

    # BLOCKS (ATTN & FFN)
    num_layers=2,                               # number of transformer blocks
    attention_types=[[["global", "local"], 1]], # (GPT-Neo-specific) global and local attention 
    num_heads=4,                                # attention heads
    window_size=256,                            # (GPT-Neo-specific) for local attention 
    intermediate_size=1024,                     # size of 'up-projection' layer in FFN

    pad_token_id = 0,           # need to specify this for tokenizer interop between models
)

config_gpt = GPTNeoConfig(**config_gpt)

### Implement KAN instead of MLP

In [ ]:
from kan import KanLayer as BaseKANLayer
import torch
from torch import nn

'''
class GPTNeoMLP(nn.Module):
    def __init__(self, intermediate_size, config):  # in MLP: intermediate_size= 4 * hidden_size
        super().__init__()
        embed_dim = config.hidden_size
        self.c_fc = nn.Linear(embed_dim, intermediate_size)
        self.c_proj = nn.Linear(intermediate_size, embed_dim)
        self.act = ACT2FN[config.activation_function]
        self.dropout = nn.Dropout(float(config.resid_dropout))

    def forward(self, hidden_states):
        hidden_states = self.c_fc(hidden_states)
        hidden_states = self.act(hidden_states)
        hidden_states = self.c_proj(hidden_states)
        hidden_states = self.dropout(hidden_states)
        return hidden_states
'''

'''
self.mlp = GPTNeoMLP(inner_dim, config)
'''

class KANLayer(nn.Module):
    def __init__(self, config):
        super(KANLayer, self).__init__()
        self.in_dim = config.hidden_size
        self.out_dim = config.hidden_size  # Keeping output size same as input
        self.kan_internal = BaseKANLayer(
            in_dim=self.in_dim,
            out_dim=self.out_dim,
            num=5,  # Example parameter
            k=3,  # Example parameter
            noise_scale=0.1,  # Example parameter
            scale_base=1.0,  # Adjust as necessary
            scale_sp=1.0,  # Adjust as necessary
            base_fun=nn.SiLU(),  # Example activation function used within KAN
            grid_eps=0.02,  # Example parameter
            grid_range=[-1, 1],  # Example parameter range
            sp_trainable=True,
            sb_trainable=True,
            device=config.device
        )
        self.dropout = nn.Dropout(config.resid_dropout)

    def forward(self, x):
        return self.


#### Add custom KAN implementation to GPT-Neo

In [10]:
class CustomGPTNeoForCausalLM(GPTNeoForCausalLM):
    def __init__(self, config, mlp_implementation=None):
        super().__init__(config)
        self.attn_implementation = mlp_implementation
        
        if mlp_implementation == "KAN":
            # Override MLP with KAN in each transformer block
            for block in self.transformer.h:
                block.mlp = KANLayer()
set_seed(42)

gpt = GPTNeoForCausalLM(config=config_gpt)

print(f'''
    This GPT has {gpt.num_parameters():,} parameters,
    ''')

# gpt, rob # uncomment to see model architecture


    This GPT has 9,677,048 parameters,
    


#### Pre-Processing
All experiments use the same tokenizer, so in theory, we only need to preprocess our data once. This means you can subsequently skip this CPU-intensive task on the DelftBlue GPU nodes. 
(except if you are experimenting with different context lengths, in that case you will need to re-tokenise to recover potentially truncated inputs; see below).

When running experiments on delftblue, it may be fastest to load (parts of) the dataset into memory, as the cluster has pretty slow IO. See `datasets`, specifically [`load_dataset`'s `keep_in_memory` attribute](https://huggingface.co/docs/datasets/en/package_reference/loading_methods#datasets.load_dataset). 

In [ ]:
# import tqdm 
# from datasets import load_dataset, DatasetDict
# 
# dataset = load_dataset('roneneldan/tinystories', num_proc=16)
# 
# n_words = 0 
# for story in tqdm.tqdm(dataset['train']['text']):    # tqdm is used for progress bars around iterables
#     n_words += len(story.split())
# 
# print(f'''
#     Train dataset has {len(dataset['train']['text']):,} samples, 
#     totalling {n_words:,} words (avg. {n_words/len(dataset['train']['text']):,.1f} per story).
#     ''')

#### Tokenisation
Two things happen during tokenisation, of which the latter is optional:

1. Map (sub-)words to integers $ \in [0, 10000]$.
2. Truncate/pad each input as necessary to fit within `hidden_size`. 

All it does is map frequent character strings (i.e. tokens) to integers, which it can then use to look up the corresponding embedding vector for that integer index.

The intuition is that by assigning each token its own embedding (vector of `1 x hidden_size`) in the transformer, will help it learn semantical understanding of these tokens. E.g. say that you have two tokens 'about' and 'around', the transformer should learn similar embedding vectors for each. 

In [11]:
from transformers import PreTrainedTokenizer, PretrainedConfig

def get_tokenizer_for_config(Tok: PreTrainedTokenizer, config: PretrainedConfig):

    tokenizer = Tok.from_pretrained(
        '10k-tok',                                         # our custom tokenizer
        model_max_length=config.max_position_embeddings    # sequence length (context window)
    )

    # we're using our special tokenizer with only 10'000 tokens instead of 50'256
    assert tokenizer.vocab_size == config.vocab_size

    print(f'padding token is {tokenizer.pad_token}')
    print(f'padding token in config: {config.pad_token_id}, in tokeniser: {tokenizer.pad_token_id}')
    
    return tokenizer 

tok_gpt = get_tokenizer_for_config(GPT2TokenizerFast, config_gpt)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RobertaTokenizer'. 
The class this function is called from is 'GPT2TokenizerFast'.


padding token is <pad>
padding token in config: 0, in tokeniser: 0


In [ ]:
# NOTE: uncomment to tokenize the dataset (both tokenizers should give identical results)
# tokenized_gpt = dataset.map(
#     lambda x: tok_gpt(x['text'], truncation=True, padding='max_length'), 
#     batched=True, num_proc=None, batch_size=1_000)
# tokenized_gpt.save_to_disk('./tokenized_dataset_small', num_proc=None)
# # sanity check that the tokenisers are indeed identical 
# tokenized_rob = dataset.map(
#     lambda x: tok_rob(x['text'], truncation=True, padding='max_length'), 
#     batched=True, num_proc=64, batch_size=1_000)

# for split in ['train', 'validation']:
#     for sample_gpt, sample_rob in tqdm.tqdm(zip(tokenized_gpt[split], tokenized_rob[split])):
#         assert sample_gpt == sample_rob

In [12]:
from datasets import load_from_disk 
tokenized_dataset = load_from_disk(f'./tokenized_dataset_small') if small_dataset else load_from_disk('./tokenized_dataset')

train_dataset = tokenized_dataset['train']
eval_dataset  = tokenized_dataset['validation']

assert len(tokenized_dataset['train'][0]['input_ids']) == config_gpt.max_position_embeddings
tokenized_dataset['train'][0]['input_ids'][-10:]
# should be pad tokens (0), given that most short stories are <512 tokens

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

#### Training
Before we get started, you may want to specify which GPU to use. See the first cell in this notebook; make sure to run it before anything else. 

Huggingface provides some powerful (and often confusingly long) APIs for model training. The `TrainingArguments` specifies our hyperparameters, which are used by the `Trainer` taking in the remaining objects (like `model`, `tokenizer`, and `train_dataset`). Specifically:

- `learning_rate` and `num_train_epochs` determine how much the model learns. A higher rate is faster, but more unstable. More epochs (entire passes over the dataset) yields incrementally better results, at the cost of more training time. 
- Batch sizes determine how many samples the model sees in *parallel*. Given `gradient_accumulation_steps=1` and a `batch_size=8`, the model will backpropagate the average loss of 8 samples; if `batch_size=1`, it will average the loss of `gradient_accumulation_steps` samples. It is important to make sure the backpropagated loss is averaged over the same number of samples, when comparing models. 

- `data_collator` batches (and optionally, pads) the input for the model. We have already padded in our `tokenized_dataset`, and leaving this argument empty will automatically batch the inputs. So why do we need it? 

    Glad you asked. This has to do with how the loss is computed in causal language modelling. In our case, we try to predict $p(y | x)$, where $x$ is an input sequence of tokens, and $y$ is the next token following that sequence. Our model, unaware of the target token $y$, outputs $\hat y$. 
    
    For `Trainer` to compute the (cross-entropy) loss, we need to provide it with both $y$ and $\hat y$. The `DataCollatorForLanguageModeling` knows this, and provides the next token $y$ as a separate part of the input, to the `Trainer`.

    The loss is the backbone of backpropagation, which we need to actually improve our model. If this is confusing, please re-watch Karpathy's GPT tutorial. 

If you prefer to write the training loop yourself, check out HF' `run_clm_no_trainer.py` scripts. (`run_mlm_no_trainer.py` for RoBERTa-style masked-language modelling, as opposed to causal language modelling). This can be useful to give you better control over which devices are used for training. 

In [14]:
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling

def get_hyperparameters(model, dataset):
    ''' common hyperparameters to give to the trainer '''

    # TRAINING HYPERPARAMETERS 
    batch_size = 16                  # TinyStories uses 80, but I am training locally on my poor M1 Air
    num_train_epochs = 1             # TinyStories doesn't mention
    gradient_accumulation_steps = 16 # TinyStories uses 16

    lr = 5e-4                        # TinyStories uses 5e-4, higher values better for small models

    # future you will thank you for descriptive model names
    # TODO: customise this name such that every model you train has a unique identifier!
    config      = model.config 
    model_name  = '-'.join([
        'GPT' if isinstance(model, GPTNeoForCausalLM) else 'BERT',
        f'{model.num_parameters()//1e6:.1f}M',
        f'{config.num_layers if isinstance(model, GPTNeoForCausalLM) else config.num_hidden_layers}L', 
        f'{config.num_heads if isinstance(model, GPTNeoForCausalLM) else config.num_attention_heads}H', 
        f'{config.hidden_size}C',
        f'{config.intermediate_size}I'
    ])

    _train_steps = len(dataset) // (batch_size * gradient_accumulation_steps)
    eval_steps = _train_steps // 10 # evaluate every 10% of training steps

    return dict(
        model_name = model_name,
        batch_size = batch_size, 
        num_train_epochs = num_train_epochs,
        gradient_accumulation_steps = gradient_accumulation_steps,
        lr = lr,
        eval_steps = eval_steps
    )

params_gpt = get_hyperparameters(gpt, train_dataset)

In [26]:
def get_trainer(
        model, tokenizer, train_dataset, eval_dataset, output_dir,
        model_name, batch_size, num_train_epochs, gradient_accumulation_steps, lr, eval_steps):
    ''' more general training arguments you likely want to keep fixed'''

    training_args = TrainingArguments(

        seed       = 42,
        use_cpu    = False, # use GPU if available (not necessarily faster on laptops, but Apple's MPS have good support)

        output_dir = os.path.join(output_dir, model_name),

        # NOTE: training params
        learning_rate    = lr,
        num_train_epochs = num_train_epochs,
        # Use a smaller batch size to fit into GPU RAM. 
        per_device_train_batch_size = batch_size,
        per_device_eval_batch_size  = batch_size,
        # You should aim to have the same amount of samples per acc step, in all of your experiments!
        # so, if you increase batch_size, decrease gradient_accumulation_steps by the same factor.
        gradient_accumulation_steps = gradient_accumulation_steps,

        # NOTE: Evaluation params
        # wandb is great for tracking experiments, it will even (try to) save your code nowadays
        evaluation_strategy = 'steps',
        eval_steps = eval_steps,
        save_steps = eval_steps,

        logging_first_step=True,
        logging_steps=100,
        report_to  = 'wandb',
    )

    trainer = Trainer(
        model = model, 
        args = training_args, 
        train_dataset = train_dataset, 
        eval_dataset = eval_dataset,
        data_collator = DataCollatorForLanguageModeling(
            tokenizer, mlm=False),
    )

    # print amount of training steps, and how often the model is evaluated
    print(f'''
    Retrieving Trainer for \033[1m{model_name}\033[0m
        training for {num_train_epochs} epochs, {len(train_dataset)} samples
        {batch_size} batch size, {gradient_accumulation_steps} accumulation steps
        gives {len(train_dataset)//(batch_size * gradient_accumulation_steps)} training steps.
        Evaluating every {eval_steps} steps, {len(eval_dataset)} samples 
        ''')

    return trainer

In [28]:
out_dir = './results/models_baseline_small/' 

trainer_gpt = get_trainer(gpt, tok_gpt, train_dataset, eval_dataset, out_dir, **params_gpt)


    Retrieving Trainer for GPT-9.0M-2L-4H-516C-1024I
        training for 1 epochs, 1000 samples
        16 batch size, 16 accumulation steps
        gives 3 training steps.
        Evaluating every 0 steps, 100 samples 
        


Finally, we can train. 

This configuration takes ≤24hr to pre-train on my M1 Macbook Air with 16GB RAM. Python takes ≤4GB VRAM at a `batch_size=16` and ≤11GB at `batch_size=64`, though they take the same amount of time to train - likely because this processor is not designed to move that much data in and out of RAM constantly. And tbh, the GPU be lacking. If you decide to go the local-training-route, consider [chai](https://github.com/lvillani/chai) to keep your (Apple) laptop awake – there's probably a windows/linux equivalent too. 

In [29]:
def do_train(trainer: Trainer, name: str, out_dir: str): 

    wandb.init(project='tiny-transformers', name=name, group='baseline', config=trainer.args)
    trainer.train()
    trainer.save_model(os.path.join(out_dir, name))

In [30]:
# words vs. tokens 
len(train_dataset['text'][11]), len(train_dataset[11]['input_ids'])

(471, 512)

In [31]:
do_train(trainer_gpt, params_gpt['model_name'], out_dir) 

  0%|          | 0/3 [00:00<?, ?it/s]

{'loss': 9.3093, 'grad_norm': 2.560922384262085, 'learning_rate': 0.0003333333333333333, 'epoch': 0.25}
{'train_runtime': 29.4856, 'train_samples_per_second': 33.915, 'train_steps_per_second': 0.102, 'train_loss': 8.672565778096518, 'epoch': 0.76}


#### Using your Pre-Trained Model 
Try out your own model on some prompts!

In [ ]:
# I made a small function for pretty printing into paragraphs with line wrapping for readability
import textwrap 
w = textwrap.TextWrapper(replace_whitespace=False, break_long_words=False, width=60, initial_indent='   ', subsequent_indent='  ')
def see(text): print('\n\033[3m' + '\n\n'.join(['\n'.join(w.wrap(line))
                 for line in text.splitlines() if line.strip() != '']) + '\033[0m\n')

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = 'GPT-9.0M-2L-4H-516C-1024I'
# model_name = 'BERT-9.0M-2L-4H-516C-1024I' # bert won't work for generation unless you fine-tune it for that task

tokenizer = AutoTokenizer.from_pretrained('10k-tok')
tokenizer.pad_token_id = tokenizer.eos_token_id
model = AutoModelForCausalLM.from_pretrained(f'results/models_baseline_small/{model_name}')

#### Inference 
Let's generate a short story like the ones the model has been trained on! You'll notice that the prompt is surrounded by `<s>`, the begin-of-sequence (bos) token, and `</s>` end-of-sequence (eos) / separator (sep) token. This is from the BERT-style tokenisation, making it clear to the model where (one of several) input sequences ends. 

In [ ]:
prompt = 'Once upon a time, there was a chicken with'
input_ids = tokenizer.encode(prompt, return_tensors='pt')

output = model.generate(
    input_ids,                              # input to the model
    max_length=300,                         # maximum generation length
    eos_token_id=tokenizer.eos_token_id,    # early stopping when eos_token is output

    # num_beams=1,                            # number of beams to use in generation
    temperature=1,
)
output_text = tokenizer.decode(output[0])

# textwrap with indentation on every new paragraph
see(output_text)
output


   <s>Once upon a time, there was a chicken
  with</s>................................................................................................................................................................................................................................................................................................



tensor([[   1, 7454, 2402,  257,  640,   11,  612,  373,  257, 9015,  351,    2,
           13,   13,   13,   13,   13,   13,   13,   13,   13,   13,   13,   13,
           13,   13,   13,   13,   13,   13,   13,   13,   13,   13,   13,   13,
           13,   13,   13,   13,   13,   13,   13,   13,   13,   13,   13,   13,
           13,   13,   13,   13,   13,   13,   13,   13,   13,   13,   13,   13,
           13,   13,   13,   13,   13,   13,   13,   13,   13,   13,   13,   13,
           13,   13,   13,   13,   13,   13,   13,   13,   13,   13,   13,   13,
           13,   13,   13,   13,   13,   13,   13,   13,   13,   13,   13,   13,
           13,   13,   13,   13,   13,   13,   13,   13,   13,   13,   13,   13,
           13,   13,   13,   13,   13,   13,   13,   13,   13,   13,   13,   13,
           13,   13,   13,   13,   13,   13,   13,   13,   13,   13,   13,   13,
           13,   13,   13,   13,   13,   13,   13,   13,   13,   13,   13,   13,
           13,   13,   13,  

Sounds like me after a few beers too many, but at least the grammar is (mostly) correct. The model also learns some basic reasoning-like associations like being 'so high' allows you to see 'the whole world'. 